In [ ]:
import pandas as pd
import sqlite3
import json
import os

In [ ]:
# Define file paths
# Using absolute paths as requested, or relative if running from the same dir
# Assuming files are in the same directory for the submission script, but for now I will point to the downloads folder
ORDERS_PATH = r"c:/Users/ASUS/Downloads/Innomatics/orders.csv"
USERS_PATH = r"c:/Users/ASUS/Downloads/Innomatics/users.json"
RESTAURANTS_SQL_PATH = r"c:/Users/ASUS/Downloads/Innomatics/restaurants.sql"
OUTPUT_PATH = "final_food_delivery_dataset.csv"

In [ ]:
def load_data():
    print("Step 1: Loading CSV Data...")
    orders_df = pd.read_csv(ORDERS_PATH)
    print(f"Loaded {len(orders_df)} orders.")

    print("Step 2: Loading JSON Data...")
    users_df = pd.read_json(USERS_PATH)
    print(f"Loaded {len(users_df)} users.")

    print("Step 3: Loading SQL Data...")
    # Create an in-memory SQLite database
    conn = sqlite3.connect(':memory:')
    cursor = conn.cursor()

    # Read and execute the SQL script
    with open(RESTAURANTS_SQL_PATH, 'r') as sql_file:
        sql_script = sql_file.read()
        cursor.executescript(sql_script)

    # Load data into a DataFrame
    restaurants_df = pd.read_sql_query("SELECT * FROM restaurants", conn)
    print(f"Loaded {len(restaurants_df)} restaurants.")
    conn.close()

    return orders_df, users_df, restaurants_df

In [ ]:
def merge_data(orders_df, users_df, restaurants_df):
    print("Step 4: Merging the Data...")
    
    # Merge Orders with Users (Left Join)
    # orders.user_id -> users.user_id
    merged_df = pd.merge(orders_df, users_df, on='user_id', how='left')
    
    # Merge with Restaurants (Left Join)
    # orders.restaurant_id -> restaurants.restaurant_id
    # Note: restaurants.sql schema has 'restaurant_id'
    merged_df = pd.merge(merged_df, restaurants_df, on='restaurant_id', how='left')
    
    print(f"Merged Data Shape: {merged_df.shape}")
    return merged_df

In [ ]:
def perform_analysis(df, report_file):
    with open(report_file, 'w') as f:
        f.write("--- 3. Analysis: Understanding the Data ---\n")
        
        # 1. Order trends over time
        if 'order_date' in df.columns:
            df['order_date'] = pd.to_datetime(df['order_date'], dayfirst=True)
            df['month'] = df['order_date'].dt.to_period('M')
            monthly_orders = df.groupby('month').size()
            f.write("\n[Order Trends] Monthly Order Volume:\n")
            f.write(monthly_orders.to_string() + "\n")
        
        # 2. User behavior patterns
        user_orders = df.groupby('user_id').size()
        f.write(f"\n[User Behavior] Average Orders per User: {user_orders.mean():.2f}\n")
        
        # 3. City-wise and Cuisine-wise performance
        city_performance = df['city'].value_counts()
        f.write("\n[City-wise Performance] Orders by City:\n")
        f.write(city_performance.to_string() + "\n")
        
        cuisine_performance = df['cuisine'].value_counts()
        f.write("\n[Cuisine-wise Performance] Top Cuisines:\n")
        f.write(cuisine_performance.head().to_string() + "\n")
        
        # 4. Membership impact
        membership_spend = df.groupby('membership')['total_amount'].mean()
        f.write("\n[Membership Impact] Average Spend by Membership:\n")
        f.write(membership_spend.to_string() + "\n")
        
        # 5. Revenue distribution and Seasonality
        monthly_revenue = df.groupby('month')['total_amount'].sum()
        f.write("\n[Revenue] Monthly Revenue:\n")
        f.write(monthly_revenue.to_string() + "\n")

In [ ]:
def main():
    orders, users, restaurants = load_data()
    final_df = merge_data(orders, users, restaurants)
    
    print(f"Step 5: Saving Final Dataset to {OUTPUT_PATH}")
    final_df.to_csv(OUTPUT_PATH, index=False)
    print("File saved successfully.")
    
    report_path = "analysis_report.txt"
    perform_analysis(final_df, report_path)
    print(f"Analysis report saved to {report_path}")

In [ ]:
if __name__ == "__main__":
    main()